In [2]:
'''
@Author: Pranay Yadav, 12CS30025
@Description: Solving the Art Gallery Problem for N planar points given in CCW ordered (distinct) using the DCEL Data Structure

'''

import Tkinter
class Point:
    def __init__(self, coordinates,auxData=None):
        self.data=auxData
        self.coords = coordinates
        self.edge = None
        self.ear = False
        self.next = None
        self.prev = None
        self.color= -1
    def ___str___(self):
        return str(self.ID)
    def __getitem__(self,key):
        return self.coords[key]
    def scale(self, k1, k2):
        self.coords = list(self.coords)
        self.coords[0] = int(self.coords[0] * k1)
        self.coords[1] = int(self.coords[1] * k2)
        self.coords = tuple(self.coords)
    def __hash__(self):
        return hash(id(self))
    def getData(self):
        return self.data
    def setData(self, auxData):
        self.data = auxData
    def getCoords(self):
        return Point(self.coords)
    def setCoords(self):
        self.coords = coordinates
    def getOutgoingEdges(self):
        visited = set()
        out = []
        here = self.edge
        while here and here not in visited:
            out.append(here)
            visited.add(here)
            temp = here.getTwin()
            if temp:
                here = temp.getNext()
            else:
                here = None
        return out
    def getIncidentEdge(self):
        return self.edge
    def setIncidentEdge(self, edge):
        self.edge = edge
    def __repr__(self):
        return 'DCEL.Point with coordnates (' + str(self.coords[0])+','+str(self.coords[1])+')'

class Edge:
    def __init__(self, auxData=None):
        self.data = auxData
        self.twin = None
        self.origin = None
        self.face = None
        self.next = None
        self.prev = None
    def __hash__(self):
        return hash(id(self))
    def getTwin(self):
        return self.twin
    def setTwin(self, twin):
        self.twin = twin
    def getData(self):
        return self.data
    def setData(self, auxData):
        self.data = auxData
    def getNext(self):
        return self.next
    def setNext(self, edge):
        self.next = edge
    def getOrigin(self):
        return self.origin
    def setOrigin(self, v):
        self.origin = v
    def getPrev(self):
        return self.prev
    def setPrev(self, edge):
        self.prev = edge
    def getDest(self):
        return self.twin.origin
    def getFace(self):
        return self.face
    def getFaceBoundary(self):
        visited = set()
        bound = []
        here = self
        while here and here not in visited:
            bound.append(here)
            visited.add(here)
            here = here.getNext()
        return bound
    def setFace(self, face):
        self.face = face
    def clone(self):
        c = Edge()
        c.data,c.twin,c.origin,c.face,c.next,c.prev = self.data,self.twin,self.origin,self.face,self.next,self.prev
    def __repr__(self):
        return 'DCEL.Edge from Origin: DCEL.Point with coordinates (' + str(self.getOrigin().coords[0])+','+str(self.getOrigin().coords[1])+')' + '\nDestination: DCEL.Point with coordinates (' + str(self.getDest().coords[0])+','+str(self.getDest().coords[1])+')'

class Face:
    def __init__(self, auxData=None):
        self.data = auxData
        self.outer = None
        self.inner = set()
        self.isolated = set()
    def __hash__(self):
        return hash(id(self))
    def getOuterComponent(self):
        return self.outer
    def setOuterComponent(self, edge):
        self.outer = edge
    def getData(self):
        return self.data
    def setData(self, auxData):
        self.data = auxData
    def getOuterBoundary(self):
        if self.outer:
            return self.outer.getFaceBoundary()
        else:
            return []
    def getOuterBoundaryCoords(self):
        original_pts = self.getOuterBoundary()
        return [x.origin.coords for x in original_pts]
    def getInnerComponents(self):
        return list(self.inner)
    def addInnerComponent(self, edge):
        self.inner.add(edge)
    def removeInnerComponent(self, edge):
        self.inner.discard(edge)
    def removeIsolatedVertex(self,Point):
        self.isolated.discard(Point)
    def getIsolatedVertices(self):
        return list(self.isolated)
    def addIsolatedVertex(self,Point):
        self.isolated.add(Point)

class DCEL:
    def __init__(self):
        self.exterior = Face()
    def getExteriorFace(self):
        return self.exterior
    def getFaces(self):
        result = []
        known = set()
        temp = []
        temp.append(self.exterior)
        known.add(self.exterior)
        while temp:
            f = temp.pop(0)
            result.append(f)
            for e in f.getOuterBoundary():
                nb = e.getTwin().getFace()
                if nb and nb not in known:
                    known.add(nb)
                    temp.append(nb)
            for inner in f.getInnerComponents():
                for e in inner.getFaceBoundary():
                    nb = e.getTwin().getFace()
                    if nb and nb not in known:
                        known.add(nb)
                        temp.append(nb)
        return result

    def getEdges(self):
        edges = set()
        for f in self.getFaces():
            edges.update(f.getOuterBoundary())
            for inner in f.getInnerComponents():
                edges.update(inner.getFaceBoundary())
        return edges

    def getVertices(self):
        verts = set()
        for f in self.getFaces():
            verts.update(f.getIsolatedVertices())
            verts.update([e.getOrigin() for e in f.getOuterBoundary()])
            for inner in f.getInnerComponents():
                verts.update([e.getOrigin() for e in inner.getFaceBoundary()])
        return verts
    

def buildSimplePolygon(points):
    d = DCEL()
    if points:
        exterior = d.getExteriorFace()
        interior = Face()
        verts = []
        for p in points:
            verts.append(Point(p))
        innerEdges = []
        outerEdges = []
        for i in range(len(verts)):
            e = Edge()
            e.setOrigin(verts[i])
            verts[i].setIncidentEdge(e)
            e.setFace(interior)
            t = Edge()
            t.setOrigin(verts[(i+1)%len(verts)])
            t.setFace(exterior)
            t.setTwin(e)
            e.setTwin(t)
            innerEdges.append(e)
            outerEdges.append(t)

        for i in range(len(verts)):
            innerEdges[i].setNext(innerEdges[(i+1)%len(verts)])
            innerEdges[i].setPrev(innerEdges[i-1])
            outerEdges[i].setNext(outerEdges[i-1])
            outerEdges[i].setPrev(outerEdges[(i+1)%len(verts)])
        interior.setOuterComponent(innerEdges[0])
        exterior.addInnerComponent(outerEdges[0])   
    return d


class Triangulation:
    def __init__(self, head = None, size = 0):
        self.HEAD = head
        self.SIZE = size
    def cloneLinkedList(self):
        cursor = self.HEAD
        newHead = Point(cursor)
        newHead.ear = cursor.ear
        cursor = cursor.next
        newCursor = newHead
        while cursor.coords is not self.HEAD.coords:
            newPoint = Point(cursor)
            newPoint.ear = cursor.ear
            newCursor.next = newPoint
            newPoint.prev = newCursor
            cursor = cursor.next
            newCursor = newPoint 
        newHead.prev = newCursor
        newCursor.next = newHead
        return newHead, self.SIZE

    def Area(self, a, b, c):
        ap=a.coords
        bp=b.coords
        cp=c.coords
        a1 = (bp[0] - ap[0]) * 1.0 * (cp[1] - ap[1])
        a2 = (cp[0] - ap[0]) * 1.0 * (bp[1] - ap[1])
        area2 = a1 - a2
        if area2 > 0.5:
            return 1
        if area2 < -0.5:
            return -1
        return 0


    def AreaDet(self, a, b, c):
        return ((b.coords[0] - a.coords[0]) * (c.coords[1] - a.coords[1])) - ((c.coords[0] - a.coords[0]) * (b.coords[1] - a.coords[1]))

    def boolXOR(self, x, y):
        return x is not y

    def liesLeft(self, a,b,c):
        return self.Area(a,b,c) > 0

    def liesLeftOrOn(self, a,b,c):
        return self.Area(a,b,c) >= 0

    def Collinear(self, a,b,c):
        return self.Area(a,b,c) == 0

    def Between(self, a, b, c):
        if not self.Collinear(a, b, c):
            return False
        if a.coords[0] != b.coords[0]:
            return (a.coords[0] <= c.coords[0] and c.coords[0] <= b.coords[0]) or (a.coords[0] >= c.coords[0] and c.coords[0] >= b.coords[0])
        else:
            return (a.coords[1] <= c.coords[1] and c.coords[1] <= b.coords[1]) or (a.coords[1] >= c.coords[1] and c.coords[1] >= b.coords[1])

    def Intersect(self, a, b, c , d):
        if self.intersectPr(a, b, c, d):
            return True
        elif self.Between(a, b, c) or self.Between(a, b, d) or self.Between(c,d,a) or self.Between(c, d, b):
            return True
        return False

    def Dgnlize(self, a, b, HEAD):
        c = HEAD
        c1 = None
        while True:
            c1 = c.next
            if (c is not a) and (c1 is not a) and (c is not b) and (c1 is not b) and self.Intersect(a, b, c, c1):
                return False
            c = c.next
            if c is HEAD:
                break
        return True

    def EarInit(self, HEAD):
        v0 = None
        v1 = None
        v2 = None
        v1 = HEAD
        while True:
            v2 = v1.next
            v0 = v1.prev
            v1.ear = self.Dgnl(v0, v2, HEAD)
            v1 = v1.next
            if v1 is HEAD:
                break

    def Triangulate(self):
        v0 = None
        v1 = None
        v2 = None
        v3 = None
        v4 = None
        earfound = False
        returnlist = []
        self.EarInit(self.HEAD)
        HEAD, n = self.cloneLinkedList()
        while n > 3:
            v2 = HEAD
            earfound = False
            while True:
                if v2.ear:
                    earfound = True
                    v3 = v2.next
                    v4 = v3.next
                    v1 = v2.prev
                    v0 = v1.prev
                    tri = [v1, v2, v3]
                    returnlist.append(tri)
                    v1.ear = self.Dgnl(v0, v3, HEAD)
                    v3.ear = self.Dgnl(v1, v4, HEAD)
                    v1.next = v3
                    v3.prev = v1
                    HEAD = v3
                    n -= 1
                    if n == 3:
                        v2 = HEAD
                        v1 = v2.prev
                        v3 = v2.next
                        tri = [v1, v2, v3]
                        returnlist.append(tri)
                        return returnlist
                    break
                v2 = v2.next
                if v2 is HEAD:
                    break
            if not earfound:
                break
        return returnlist

    def iCON(self, a, b):
        a0 = None
        a1 = None
        a1 = a.next
        a0 = a.prev
        if self.liesLeftOrOn(a, a1, a0):
            return self.liesLeft(a,b,a0) and self.liesLeft(b,a,a1)
        return not (self.liesLeftOrOn(a,b,a1) and self.liesLeft(b,a,a0))

    def intersectPr(self, a, b, c, d):
        if self.Collinear(a,b,c) or self.Collinear(a,b,d) or self.Collinear(c,d,a) or self.Collinear(c,d,b):
            return False
        return self.boolXOR(self.liesLeft(a,b,c), self.liesLeft(a,b,d)) and self.boolXOR(self.liesLeft(c,d,a) , self.liesLeft(c,d,b))

    def Dgnl(self, a, b, HEAD):
        return self.iCON(a,b) and self.iCON(b, a) and self.Dgnlize(a,b, HEAD)

    def drawPolygon(self, canvas):
        cursor = self.HEAD
        while True:
            x1 = cursor.coords[0]
            y1 = cursor.coords[1]
            x2 = cursor.next.coords[0]
            y2 = cursor.next.coords[1]
            #label = Tkinter.Label(canvas, text = "("+str(cursor.coords[0])+","+str(cursor.coords[1])+")", font = "Times 6")
            #label.place(x = x1 + 4, y = 700 - (y1 + 4))
            canvas.create_line(x1, 700 - y1, x2, 700 - y2, width = 2.0, fill = 'black')
            canvas.create_oval(x1 - 4, 700 - (y1 - 4), x1 + 4, 700 - (y1 + 4), fill = 'black')
            cursor = cursor.next
            if cursor.coords is self.HEAD.coords:
                break

    def drawSubPolygon(self, canvas):
        cursor = self.HEAD
        while True:
            x1 = cursor.coords[0]
            y1 = cursor.coords[1]
            x2 = cursor.next.coords[0]
            y2 = cursor.next.coords[1]
            #label = Tkinter.Label(canvas, text = "("+str(cursor.coords[0])+","+str(cursor.coords[1])+")", font = "Times 6")
            #label.place(x = x1 + 4, y = 700 - (y1 + 4))
            canvas.create_line(x1, 700 - y1, x2, 700 - y2, width = 1.0, fill = 'red')
            canvas.create_oval(x1 - 4, 700 - (y1 - 4), x1 + 4, 700 - (y1 + 4), fill = 'black')
            cursor = cursor.next
            if cursor.coords is self.HEAD.coords:
                break        

    def drawTriangles(self, canvas, triangles):
        pointlist = []
        cursor = self.HEAD
        while True:
            pointlist.append(cursor)
            cursor = cursor.next
            if cursor.coords is self.HEAD.coords:
                break
        if len(triangles) > 0:
            for t in triangles:
                canvas.create_line(t[0].coords[0], 700 - t[0].coords[1], t[1].coords[0], 700 - t[1].coords[1],width = 1.0, fill = 'red')
                canvas.create_line(t[0].coords[0], 700 - t[0].coords[1], t[2].coords[0], 700 - t[2].coords[1],width = 1.0, fill = 'red')
                canvas.create_line(t[2].coords[0], 700 - t[2].coords[1], t[1].coords[0], 700 - t[1].coords[1],width = 1.0, fill = 'red')

    def scale(self, uniform = False):
        xmin = 1000000000 # Initialize to some huge numbers
        ymin = 1000000000
        cursor = self.HEAD
        while True:
            if cursor.coords[0] < xmin:
                xmin = cursor.coords[0]
            if cursor.coords[1] < ymin:
                ymin = cursor.coords[1]
            cursor = cursor.next
            if cursor.coords is self.HEAD.coords:
                break

        cursor = self.HEAD
        while True:
            cursor.coords = list(cursor.coords)
            cursor.coords[0] = cursor.coords[0] - xmin + 10
            cursor.coords[1] = cursor.coords[1] - ymin + 10
            cursor = cursor.next
            if cursor.coords is self.HEAD.coords:
                break

        xmin = 1000000000
        ymin = 1000000000
        xmax = 0
        ymax = 0
        cursor = self.HEAD
        while True:
            if cursor.coords[0] < xmin:
                xmin = cursor.coords[0]
            if cursor.coords[1] < ymin:
                ymin = cursor.coords[1]
            if cursor.coords[0] > xmax:
                xmax = cursor.coords[0]
            if cursor.coords[1] > ymax:
                ymax = cursor.coords[1]
            cursor = cursor.next
            if cursor.coords is self.HEAD.coords:
                break

        k1 = 990.0 / xmax
        k2 = 550.0 / ymax
        if uniform:
            if k1 < k2:
                k2 = k1
            else:
                k1 = k2
        cursor = self.HEAD
        while True:
            cursor.scale(k1, k2)
            cursor = cursor.next
            if cursor.coords is self.HEAD.coords:
                break

def dualgraph(map_points,listTriangle):
    dvert={i:listTriangle[i] for i in range(0,len(listTriangle))}
    dedge={}
    for i in range(0,len(listTriangle)):
        j=i+1
        for j in range(0,len(listTriangle)):
            if len(list(set(listTriangle[i])&set(listTriangle[j]))) > 1:
                if i in dedge and j not in dedge[i] and i is not j:
                    dedge[i].append(j)
                elif i not in dedge and i is not j:
                    dedge[i]=[j]	
                if j in dedge and i not in dedge[j] and i is not j:
                    dedge[j].append(i)
                elif j not in dedge and i is not j:
                    dedge[j]=[i]	
    return dvert,dedge					

def DFS(s,vdual,edual):
    visited, stack = set(), [s]
    while stack:
        vertex = stack.pop()
        if vertex not in visited:
            colorsum = vdual[vertex][0].color+vdual[vertex][1].color+vdual[vertex][2].color
            if colorsum<3:
                print ("Triangle #"+str(vertex)+" has one vertex uncolored!!!")
                if vdual[vertex][0].color is -1:
                    vdual[vertex][0].color = 3-vdual[vertex][1].color-vdual[vertex][2].color
                    print ("Triangle #"+str(vertex)+" Vertex #0 found uncolored!!! Now colored to "+str(vdual[vertex][0].color))
                elif vdual[vertex][1].color is -1:
                    vdual[vertex][1].color = 3-vdual[vertex][0].color-vdual[vertex][2].color
                    print ("Triangle #"+str(vertex)+" Vertex #1 found uncolored!!! Now colored to "+str(vdual[vertex][1].color))
                elif vdual[vertex][2].color is -1:
                    vdual[vertex][2].color = 3-vdual[vertex][1].color-vdual[vertex][0].color		 
                    print ("Triangle #"+str(vertex)+" Vertex #2 found uncolored!!! Now colored to "+str(vdual[vertex][2].color))
            visited.add(vertex)
            stack.extend(set(edual[vertex]) - visited)
    return visited

def colorize(map_points,listTriangle,vdual,edual):
    ear_vertex=None
    ear_tri=None
    key=None
    for ear_vertex in map_points:
        if map_points[ear_vertex].ear:
            break;

    for ear_tri in listTriangle:
        if ear_vertex in ear_tri:
            break

    for key in vdual:
        if vdual[key] is ear_vertex:
            break

    print("############################# INITIAL COLORING OF ONE TRIANGLE ##################################")
    print ("Triangle #"+str(key)+" Vertex #0 colored to 0")
    ear_tri[0].color = 0
    print ("Triangle #"+str(key)+" Vertex #1 colored to 1")
    ear_tri[1].color = 1
    print ("Triangle #"+str(key)+" Vertex #2 colored to 2")
    ear_tri[2].color = 2
    print("############################# GOING TO COLOR REMAINING TRIANGLES ###############################")
    DFS(key,vdual,edual)
    output,col = findMinColor(vdual.values())
    print("Guards are colored "+str(col-1))

def findMinColor(triangles):
    rcount,gcount,bcount=0,0,0
    r,g,b=[],[],[]
    out= set()
    for t in triangles:
        for it in t:
            if it not in out:
                if it.color is 0:
                    rcount+=1
                    r.append(it)
                elif it.color is 1:
                    gcount+=1
                    g.append(it)
                elif it.color is 2:
                    bcount+=1
                    b.append(it)	
                out.add(it)	
    if rcount is gcount and rcount is bcount:
        return r,rcount
    if rcount<=gcount and rcount<=bcount:
        return r,rcount
    if gcount<=rcount and gcount<=bcount:
        return g,gcount
    if bcount<=rcount and bcount<=gcount:
        return b,bcount


In [3]:
# divide polygon in two // insert diagonal

def insertDgnl(d, p1, p2):
    pointlist1 = []
    pointlist2 = []
    original_pts = d.getFaces()[1].getOuterBoundaryCoords()
#     original_pts = d.getFaces()[1].getOuterBoundary()
#     original_pts = [x.origin.coords for x in original_pts]
#     print "d:",original_pts

    if (p1 in original_pts and p2 in original_pts) and p1!=p2:
        tmp1 = min(original_pts.index(p1), original_pts.index(p2))
        tmp2 = max(original_pts.index(p1), original_pts.index(p2))
#         print original_pts.index(p1)
#         print original_pts.index(p2)
        pointlist1 = original_pts[tmp1:(tmp2+1)]
#         print pointlist1
        pointlist2 = original_pts[tmp2:]+original_pts[:(tmp1+1)]
#         print pointlist2
        d1 = buildSimplePolygon(pointlist1)
        d2 = buildSimplePolygon(pointlist2)
        return [d1,d2]
    return [d]

# print insertDgnl(d,p1,p2)

In [4]:
# divide polygon in many // insert list of diagonals

def insertDgnls(d, dgnls):
    ngons = [d]
    while dgnls != []:
        nxt = dgnls.pop(0)
        ngons = [insertDgnl(x, nxt[0], nxt[1]) for x in ngons]
        ngons = [ngon for lngon in ngons for ngon in lngon]
    return ngons

pl = insertDgnls(d, [((4, 6), (2, 9)), ((1, 5), (4, 6)), ((3, 2), (1, 5))])
print pl

[[x.origin for x in f.getFaces()[1].getOuterBoundary()] for f in pl]

NameError: name 'd' is not defined

In [5]:
class trapEdge(object):
    def __init__(self,a,b,s,l,r):
        self.left = a
        self.right = b
        self.pivot = s
        self.le = l
        self.re = r

class point(object):
    def __init__(self, a, b):
        self.x = a
        self.y = b
def onSegment(p,q,r):
    if (q.x <= max(p.x, r.x) and q.x >= min(p.x, r.x) and q.y <= max(p.y, r.y) and q.y >= min(p.y, r.y)):
        return True
    return False
def orientation(p,q,r):
    val = (q.y - p.y) * (r.x - q.x) -(q.x - p.x) * (r.y - q.y)
    if (val == 0):
        return 0
    if(val>0):
        return 1
    return 2
def doIntersect(p1,q1,p2,q2):
    o1 = orientation(p1, q1, p2);
    o2 = orientation(p1, q1, q2);
    o3 = orientation(p2, q2, p1);
    o4 = orientation(p2, q2, q1);
    A,B,C,D = p1,q1,p2,q2
    a1 = B.y - A.y
    b1 = A.x - B.x
    c1 = a1*(A.x) + b1*(A.y)
    a2 = D.y - C.y
    b2 = C.x - D.x
    c2 = a2*(C.x)+ b2*(C.y)
    determinant = a1*b2 - a2*b1
    if(determinant == 0):
        return False
    if (o1 != o2 and o3 != o4):
        return True
    if (o1 == 0 and onSegment(p1, p2, q1)):
        return True
    if (o2 == 0 and onSegment(p1, q2, q1)):
        return True
    if (o3 == 0 and onSegment(p2, p1, q2)):
        return True
    if (o4 == 0 and onSegment(p2, q1, q2)):
        return True
    return False

def findIt(A,B,C,D):
    a1 = B.y - A.y
    b1 = A.x - B.x
    c1 = a1*(A.x) + b1*(A.y)
    a2 = D.y - C.y
    b2 = C.x - D.x
    c2 = a2*(C.x)+ b2*(C.y)
    determinant = a1*b2 - a2*b1
    x = (b2*c1 - b1*c2)/determinant
    y = (a1*c2 - a2*c1)/determinant
    return (x, y)

def findIntersections(lines, hlines):
    res = {}
    for hline in hlines:
        p1 = point(hline[0],hline[1])
        q1 = point(hline[2],hline[3])
        for line in lines:
            p2 = point(line[0][0],line[0][1])
            q2 = point(line[0][2],line[0][3])
            if(doIntersect(p1,q1,p2,q2)):
                res[findIt(p1,q1,p2,q2)] = line[1]
    return res

def getTrapEdges(d):
    N = len(d.getVertices())
    verts = [ list(d.getVertices())[i].coords for i in range(N) ]
    verts = zip(verts, [i for i in list(d.getVertices())])
    edges = [(verts[i][1].next.coords,verts[i][1].coords) for i in range(N) ]
    edges = zip(edges, [v[1].getOutgoingEdges()[0] for v in verts])
    verts.sort(key=lambda x: -x[0][0])
    lines = []
    temp = []
    for e in edges:
        temp = [e[0][0][0],e[0][0][1],e[0][1][0],e[0][1][1]],e[1]
        lines.append(temp)
    lines2 = []
    temp = []
    for v in verts:
        temp = verts[0][0][0],v[0][1],verts[-1][0][0],v[0][1]
        lines2.append(temp)
    res = findIntersections(lines,lines2)

    res = [[x,y,res[(x,y)]] for (x,y) in res]
    res.sort(key = lambda x: -x[1])
    ret = []
    
    for v in verts:
        templ = [(x[0],x[1],x[2]) for x in res if (x[0]<v[0][0] and x[1]==v[0][1])]
        tempr = [(x[0],x[1],x[2]) for x in res if (x[0]>v[0][0] and x[1]==v[0][1])]
        if( len(templ)%2==0 and len(tempr)%2==0 ):
            if v[1].getOutgoingEdges()[0].getTwin().origin.coords[1] < v[1].coords[1] :
                tr = trapEdge(v[0],v[0],v[1],v[1].getOutgoingEdges()[0],v[1].getOutgoingEdges()[1].getTwin())
            else:
                tr = trapEdge(v[0],v[0],v[1],v[1].getOutgoingEdges()[1].getTwin(),v[1].getOutgoingEdges()[0].getTwin())
#                 tr = trapEdge(v[0],v[0],v[1],None,None)
            ret.append(tr)
        if( len(templ)%2==1 and len(tempr)%2==1 ):
            tr = trapEdge(templ[-1][:2],tempr[0][:2],v[1],templ[-1][2],tempr[0][2])
            ret.append(tr)
        if( len(templ)%2==0 and len(tempr)%2==1 ):
            tr = trapEdge(v[0],tempr[0][:2],v[1],v[1].getIncidentEdge(), tempr[0][2])
            ret.append(tr)
        if( len(templ)%2==1 and len(tempr)%2==0 ):
            tr = trapEdge(templ[-1][:2],v[0],v[1],templ[-1][2],v[1].getOutgoingEdges()[1].getTwin())
            ret.append(tr)
    
    return ret

In [8]:
ret = getTrapEdges(d)
ret = sorted(ret, key=lambda x:-x.pivot.coords[1])
for x in ret:
    print x.pivot


AttributeError: 'NoneType' object has no attribute 'coords'

In [234]:
# returns list of diagonals for partioning

def monotonePartitioningDgnls(d):
    
    ret = getTrapEdges(d)
    ret = sorted(ret, key=lambda x:-x.pivot.coords[1])

    a = dict()
    b = dict()

    for x in ret:
        print "\n",x.left,x.right
        print "Pivot:",x.pivot.coords
        print "Ledge:",x.le.origin.coords,"-->",x.le.getTwin().origin.coords
        x.re = x.re.getTwin()
        print "Redge:",x.re.origin.coords,"-->",x.re.getTwin().origin.coords


        if x.pivot.coords[1] > x.re.getTwin().origin.coords[1]:
            a[x.pivot] = (x.le,x.re)
            if x.le in b:
                b[x.le].append(x.pivot)
            else:
                b[x.le] = [x.pivot]

            if x.re in b:
                b[x.re].append(x.pivot)
            else:
                b[x.re] = [x.pivot]

    for e in b:
        b[e].append(e.getTwin().origin)

    print "\n### a"    
    for (i,x) in enumerate(a):
        print
        print i,x.coords
        for e in a[x]:
            print e.origin.coords, e.getTwin().origin.coords
    print "\n### b"
    for (i,x) in enumerate(b):
        print
        print i,x.origin.coords, x.getTwin().origin.coords
        print b[x]

    dgnls = []
    print "--"

    # pt = list(a.keys())[1]

    # print "]]]]",pt.coords
    # print a[pt]
    # print [x.coords for x in b[a[pt][0]] ],b[a[pt][0]].index(pt)
    # print [x.coords for x in b[a[pt][1]] ],b[a[pt][1]].index(pt),"[[[[["

    for pt in sorted(a, key=lambda x:-x.coords[1]):
        print "\n]]]]",pt.coords
        print a[pt][0].origin.coords, a[pt][0].getTwin().origin.coords,
        print len(b[a[pt][0]]),[x.coords for x in b[a[pt][0]] ],b[a[pt][0]].index(pt)
        print a[pt][1].origin.coords, a[pt][1].getTwin().origin.coords,
        print len(b[a[pt][1]]),[x.coords for x in b[a[pt][1]] ],b[a[pt][1]].index(pt),"[[[[["

#         if not a[pt][0].origin == a[pt][1].origin:
#             print "in"
        if pt in ( a[pt][0].origin, a[pt][0].getTwin().origin ):
            dgnls.append((pt, b[a[pt][1]][b[a[pt][1]].index(pt)+1] ))
        elif pt in ( a[pt][1].origin, a[pt][1].getTwin().origin ):
            dgnls.append((pt, b[a[pt][0]][b[a[pt][0]].index(pt)+1] ))
        else:
            dgnls.append((pt,
                          min( b[a[pt][0]][b[a[pt][0]].index(pt)+1], 
                               b[a[pt][1]][b[a[pt][1]].index(pt)+1], 
                               key=lambda x:x.coords[1]
                             )
                         ))
        print "Dgnls:",[(x[0].coords,x[1].coords) for x in dgnls]

    print "ppp",[(x.origin,x.getTwin().origin) for x in d.getEdges()]
    for ww in dgnls:
        print ww in [(x.origin,x.getTwin().origin) for x in d.getEdges()]
    dgnls = list(set(dgnls)-set([(x.origin,x.getTwin().origin) for x in d.getEdges()]))
    return dgnls

monotonePartitioningDgnls(d)

AttributeError: 'NoneType' object has no attribute 'coords'

In [224]:
# triangulate Monotone Polygon // get list of diagonals
# !!!!!! sadly not yet implemented the function to check reflex angle

def orientation(p,q,r):
    p=p.coords
    q=q.coords
    r=r.coords
    val = ( q[1] - p[1] )*( r[0] - q[0] ) - ( q[0] - p[0] )* ( r[1] - q[1] )
    return -val

def reflex(p,q,r,chain = 'l'):
    if(chain == 'r'):
        if orientation(p,q,r)>=0:
            return True
        else:
            return False
    elif(chain == 'l'):
        if orientation(p,q,r)>0:
            return False
        else:
            return True
    

def triangulateMonotonePolygon(d):
    pts = [x.origin for x in d.getFaces()[1].getOuterBoundary()]
    print "Polygon Boundary:",[x.coords for x in pts]
    min_index = min(enumerate(pts), key=lambda x:x[1].coords[1])[0]
    max_index = max(enumerate(pts), key=lambda x:x[1].coords[1])[0]
    tmp1 = min(min_index,max_index)
    tmp2 = max(min_index,max_index)
    chain1 = pts[tmp1:(tmp2+1)] 
    chain2 = pts[tmp2:]+pts[:(tmp1+1)] 
    
    if(min(chain1, key=lambda x:x.coords[0]).coords[0] >min(chain2, key=lambda x:x.coords[0]).coords[0]):    # ensuring chain1 is left chain
        print "Monotone chains swapped"
        tmp = list(chain1)
        chain1 = chain2
        chain2 = tmp
    print "Left Chain     : ",[x.coords for x in chain1]
    print "Right Chain    : ",[x.coords for x in chain2]
    pts = sorted(pts, key = lambda x:-x.coords[1])
    print "\nSorted pts     : ",[x.coords for x in pts]
    print
    
    queue = []
    diagonals = []
    
    queue.append(pts[0])
    queue.append(pts[1])
    
    i = 2
    print len(pts)
    while i < (len(pts)-1):
        print "\ni =",i,";",pts[i].coords
        #process(pts[i])
        tmp1 = queue[-1] in chain1
        tmp2 = pts[i] in chain1 
        if (tmp1 and not tmp2) or (tmp2 and not tmp1):
            for qpt in queue[1:]:
                diagonals.append((pts[i], qpt))
                print "Case: a;  \nDiagonals:", [(x[0].coords,x[1].coords) for x in diagonals]
            queue = [queue[-1],pts[i]]
            print "Queue: ",[x.coords for x in queue]
        else:
            print "||||||",queue[-2],queue[-1],pts[i],"chain =", ('l' if tmp1 else 'r'),"|||||"
            print reflex(queue[-2],queue[-1],pts[i],chain = ('l' if tmp1 else 'r') )
            print orientation(queue[-2],queue[-1],pts[i] )
            if reflex(queue[-2],queue[-1],pts[i],chain = ('l' if tmp1 else 'r') ):
                queue.append(pts[i])
                print "Case: b;  \nDiagonals:",# reflex
                print [(x[0].coords,x[1].coords) for x in diagonals]
                print "Queue: ",[x.coords for x in queue]
            else:
                diagonals.append((pts[i], queue[-2]))
                print "Case: c;  \nDiagonals:", # convex
                print [(x[0].coords,x[1].coords) for x in diagonals]
                print "Queue: ",[x.coords for x in queue]
                queue.pop(-1)
                i-=1
        i+=1
    
    if len(queue)>2:
        for qpt in queue[1:-1]:
                diagonals.append((pts[i], qpt))
                print "Case: a;  \nDiagonals:", [(x[0].coords,x[1].coords) for x in diagonals]
    
    print "Queue: ",[x.coords for x in queue]
    return diagonals
            
        
triangulateMonotonePolygon(d)
# listOfTriangles = insertDgnls(d,triangulateMonotonePolygon(d))

Polygon Boundary: [(1, 8), (6, 3), (8, 4), (10, 6), (7, 11), (8, 15), (7, 16), (6, 17), (5, 19), (2, 13), (4, 12), (5, 10), (3, 9)]
Monotone chains swapped
Left Chain     :  [(5, 19), (2, 13), (4, 12), (5, 10), (3, 9), (1, 8), (6, 3)]
Right Chain    :  [(6, 3), (8, 4), (10, 6), (7, 11), (8, 15), (7, 16), (6, 17), (5, 19)]

Sorted pts     :  [(5, 19), (6, 17), (7, 16), (8, 15), (2, 13), (4, 12), (7, 11), (5, 10), (3, 9), (1, 8), (10, 6), (8, 4), (6, 3)]

13

i = 2 ; (7, 16)
|||||| DCEL.Point with coordnates (5,19) DCEL.Point with coordnates (6,17) DCEL.Point with coordnates (7,16) chain = r |||||
True
1
Case: b;  
Diagonals: []
Queue:  [(5, 19), (6, 17), (7, 16)]

i = 3 ; (8, 15)
|||||| DCEL.Point with coordnates (6,17) DCEL.Point with coordnates (7,16) DCEL.Point with coordnates (8,15) chain = r |||||
True
0
Case: b;  
Diagonals: []
Queue:  [(5, 19), (6, 17), (7, 16), (8, 15)]

i = 4 ; (2, 13)
Case: a;  
Diagonals: [((2, 13), (6, 17))]
Case: a;  
Diagonals: [((2, 13), (6, 17)), ((2, 13

[(DCEL.Point with coordnates (2,13), DCEL.Point with coordnates (6,17)),
 (DCEL.Point with coordnates (2,13), DCEL.Point with coordnates (7,16)),
 (DCEL.Point with coordnates (2,13), DCEL.Point with coordnates (8,15)),
 (DCEL.Point with coordnates (4,12), DCEL.Point with coordnates (8,15)),
 (DCEL.Point with coordnates (7,11), DCEL.Point with coordnates (4,12)),
 (DCEL.Point with coordnates (5,10), DCEL.Point with coordnates (7,11)),
 (DCEL.Point with coordnates (10,6), DCEL.Point with coordnates (5,10)),
 (DCEL.Point with coordnates (10,6), DCEL.Point with coordnates (3,9)),
 (DCEL.Point with coordnates (10,6), DCEL.Point with coordnates (1,8)),
 (DCEL.Point with coordnates (8,4), DCEL.Point with coordnates (1,8))]

In [225]:
t = triangulateMonotonePolygon(d)
listOfTriangles = insertDgnls(d,[(dg[0].coords,dg[1].coords) for dg in t])
listOfTriangles = [[x.origin for x in f.getFaces()[1].getOuterBoundary()] for f in listOfTriangles]

print
for x in enumerate(listOfTriangles):
    print "Triangle #",x[0],":",x[1]

Polygon Boundary: [(1, 8), (6, 3), (8, 4), (10, 6), (7, 11), (8, 15), (7, 16), (6, 17), (5, 19), (2, 13), (4, 12), (5, 10), (3, 9)]
Monotone chains swapped
Left Chain     :  [(5, 19), (2, 13), (4, 12), (5, 10), (3, 9), (1, 8), (6, 3)]
Right Chain    :  [(6, 3), (8, 4), (10, 6), (7, 11), (8, 15), (7, 16), (6, 17), (5, 19)]

Sorted pts     :  [(5, 19), (6, 17), (7, 16), (8, 15), (2, 13), (4, 12), (7, 11), (5, 10), (3, 9), (1, 8), (10, 6), (8, 4), (6, 3)]

13

i = 2 ; (7, 16)
|||||| DCEL.Point with coordnates (5,19) DCEL.Point with coordnates (6,17) DCEL.Point with coordnates (7,16) chain = r |||||
True
1
Case: b;  
Diagonals: []
Queue:  [(5, 19), (6, 17), (7, 16)]

i = 3 ; (8, 15)
|||||| DCEL.Point with coordnates (6,17) DCEL.Point with coordnates (7,16) DCEL.Point with coordnates (8,15) chain = r |||||
True
0
Case: b;  
Diagonals: []
Queue:  [(5, 19), (6, 17), (7, 16), (8, 15)]

i = 4 ; (2, 13)
Case: a;  
Diagonals: [((2, 13), (6, 17))]
Case: a;  
Diagonals: [((2, 13), (6, 17)), ((2, 13

In [227]:
def drawTriangles(canvas, triangles):
    if len(triangles) > 0:
        for t in triangles:
            canvas.create_line(10*t[0].coords[0], 700 - 10*t[0].coords[1], 10*t[1].coords[0], 700 - 10*t[1].coords[1],width = 1.0, fill = 'red')
            canvas.create_line(10*t[0].coords[0], 700 - 10*t[0].coords[1], 10*t[2].coords[0], 700 - 10*t[2].coords[1],width = 1.0, fill = 'red')
            canvas.create_line(10*t[2].coords[0], 700 - 10*t[2].coords[1], 10*t[1].coords[0], 700 - 10*t[1].coords[1],width = 1.0, fill = 'red')

canvas_width = 1000
canvas_height = 800
master = Tkinter.Tk()
canvas = Tkinter.Canvas(master, width=canvas_width, height=canvas_height)
canvas.pack()
drawTriangles(canvas, listOfTriangles)
Tkinter.mainloop()


In [130]:
# merge multiple polygons into one 

def mergePolygons(d,ngons):
    interior = d.getFaces()
    print interior

    

[[], [DCEL.Edge from Origin: DCEL.Point with coordinates (0,0)
Destination: DCEL.Point with coordinates (2,0), DCEL.Edge from Origin: DCEL.Point with coordinates (2,0)
Destination: DCEL.Point with coordinates (2,1), DCEL.Edge from Origin: DCEL.Point with coordinates (2,1)
Destination: DCEL.Point with coordinates (1,2), DCEL.Edge from Origin: DCEL.Point with coordinates (1,2)
Destination: DCEL.Point with coordinates (0,1), DCEL.Edge from Origin: DCEL.Point with coordinates (0,1)
Destination: DCEL.Point with coordinates (0,0)]]


In [85]:
p1 = (0,0)
p2 = (1,1)

In [90]:
# for f in d.getFaces():
#     print(f.getOuterBoundary())
    
# print d.getFaces()
pts = d.getFaces()[1].getOuterBoundary()
print pts
pts = [x.origin.coords for x in pts]
print pts

[DCEL.Edge from Origin: DCEL.Point with coordinates (0,0)
Destination: DCEL.Point with coordinates (0,1), DCEL.Edge from Origin: DCEL.Point with coordinates (0,1)
Destination: DCEL.Point with coordinates (1,1), DCEL.Edge from Origin: DCEL.Point with coordinates (1,1)
Destination: DCEL.Point with coordinates (1,0), DCEL.Edge from Origin: DCEL.Point with coordinates (1,0)
Destination: DCEL.Point with coordinates (0,0)]
[(0, 0), (0, 1), (1, 1), (1, 0)]


In [7]:
print set([x.origin.coords for x in d.getEdges()])

set([(5, 10), (10, 6), (1, 8), (5, 19), (6, 3), (2, 13), (3, 9), (7, 16), (7, 11), (6, 17), (4, 12), (8, 15), (8, 4)])


In [6]:
p=[]
N= int(input())
lines=[raw_input().strip() for i in range(0,N)]
p=[(int(lines[i].split()[0]),int(lines[i].split()[1])) for i in range(0,len(lines))]
d = buildSimplePolygon(p)
print("################################ DCEL POLYGON FORMED ###################################")
# 13
# 1 8
# 6 3
# 8 4
# 10 6
# 7 11
# 8 15
# 7 16
# 6 17
# 5 19
# 2 13
# 4 12
# 5 10
# 3 9

13
1 8
6 3
8 4
10 6
7 11
8 15
7 16
6 17
5 19
2 13
4 12
5 10
3 9
################################ DCEL POLYGON FORMED ###################################


In [145]:
map_points ={x.coords:x for x in d.getVertices()}
for i in range(1,len(p)-1):
	map_points[p[i]].next = map_points[p[i+1]]
	map_points[p[i+1]].prev = map_points[p[i]]
map_points[p[0]].prev = map_points[p[-1]]
map_points[p[0]].next = map_points[p[1]]
map_points[p[1]].prev = map_points[p[0]]
map_points[p[-1]].next = map_points[p[0]]

T = Triangulation(map_points[p[0]],len(map_points))
listTriangle = T.Triangulate()
print("################################# TRIANGULATED REGIONS ####################################")
for i,items in enumerate(listTriangle):
	print("Triangle #"+str(i))
	print(items[0])
	print(items[1])
	print(items[2])



################################# TRIANGULATED REGIONS ####################################
Triangle #0
DCEL.Point with coordnates (0,1)
DCEL.Point with coordnates (0,0)
DCEL.Point with coordnates (2,0)
Triangle #1
DCEL.Point with coordnates (0,1)
DCEL.Point with coordnates (2,0)
DCEL.Point with coordnates (2,1)
Triangle #2
DCEL.Point with coordnates (0,1)
DCEL.Point with coordnates (2,1)
DCEL.Point with coordnates (1,2)


In [148]:
print("################################ GRAPHICAL VIEW #####################################")
canvas_width = 1000
canvas_height = 600
master = Tkinter.Tk()
canvas = Tkinter.Canvas(master, width=canvas_width, height=canvas_height)
canvas.pack()
T.scale(True)
T.drawTriangles(canvas, listTriangle)
T.drawPolygon(canvas)
Tkinter.mainloop()

#for key in map_points:
# 	if map_points[key].ear:
# 		print (map_points[key])
# 		print ("is a ear!!"

################################ GRAPHICAL VIEW #####################################


In [147]:
print("############################### CONSTRUCTING DUAL GRAPH ############################")
vdual,edual = dualgraph(map_points,listTriangle)
print("############################### COLORIZING BEGINS ##################################")
colorize(map_points,listTriangle,vdual,edual)

############################### CONSTRUCTING DUAL GRAPH ############################
############################### COLORIZING BEGINS ##################################
############################# INITIAL COLORING OF ONE TRIANGLE ##################################
Triangle #2 Vertex #0 colored to 0
Triangle #2 Vertex #1 colored to 1
Triangle #2 Vertex #2 colored to 2
############################# GOING TO COLOR REMAINING TRIANGLES ###############################
Triangle #1 has one vertex uncolored!!!
Triangle #1 Vertex #1 found uncolored!!! Now colored to 2
Triangle #0 has one vertex uncolored!!!
Triangle #0 Vertex #1 found uncolored!!! Now colored to 1
Guards are colored 0
